my calibration, K

3282.61164 2024.03056 3288.85353 1453.08413 1

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2


In [2]:
def detect_markers(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    return image, corners, ids


def find_match_points(corners_1, ids_1, corners_2, ids_2):
    matching_ids = []
    matching_idx_1 = []
    matching_idx_2 = []
    matching_corners_1 = []
    matching_corners_2 = []

    for i in range(len(ids_1)):
        for j in range(len(ids_2)):
            if ids_1[i] == ids_2[j]:
                matching_ids.append(ids_1[i])

                matching_corners_1.append(corners_1[i].reshape(-1, 2))
                matching_corners_2.append(corners_2[j].reshape(-1, 2))

                matching_idx_1.append((4 * i, 4 * i + 1, 4 * i + 2, 4 * i + 3))
                matching_idx_2.append((4 * j, 4 * j + 1, 4 * j + 2, 4 * j + 3))

    # Now we have the matching IDs and corresponding corner points
    # You can use these lists for further processing with the 5-point algorithm
    matching_corners_1 = np.array(matching_corners_1).reshape(-1, 2)
    matching_corners_2 = np.array(matching_corners_2).reshape(-1, 2)

    return (
        matching_ids,
        matching_idx_1,
        matching_idx_2,
        matching_corners_1,
        matching_corners_2,
    )

In [11]:

# 시작
image_dir = "./images"
images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
images = images[7:-3]
images

['KakaoTalk_20240530_125615705_07.jpg',
 'KakaoTalk_20240530_125615705_08.jpg',
 'KakaoTalk_20240530_125615705_09.jpg',
 'KakaoTalk_20240530_125615705_10.jpg',
 'KakaoTalk_20240530_125615705_11.jpg',
 'KakaoTalk_20240530_125615705_12.jpg',
 'KakaoTalk_20240530_125615705_13.jpg',
 'KakaoTalk_20240530_125615705_14.jpg',
 'KakaoTalk_20240530_125615705_15.jpg',
 'KakaoTalk_20240530_125615705_16.jpg',
 'KakaoTalk_20240530_125615705_17.jpg',
 'KakaoTalk_20240530_125615705_18.jpg',
 'KakaoTalk_20240530_125615705_19.jpg',
 'KakaoTalk_20240530_125615705_20.jpg']

In [4]:
# sift
for image in images:
    image_path = os.path.join(image_dir, image)
    image, corners, ids = detect_markers(image_path)

    _image = image_path.split(".")[1]

    with open(f".{_image}.sift", "w") as output:
        output.write(f"{len(corners)*4} 128\n")
        
        image, corners, ids = detect_markers(image_path)
        for i, corner in enumerate(corners):
            for j, c in enumerate(corner[0]):
                output.write(f"{c[0]} {c[1]} 0 0\n")
                output.write("0 " * 128+"\n")  # Write the 128 descriptor values (all zeros in this case)
                
                # Check if this is not the last element
                # if not (i == len(corners) - 1 and j == len(corner[0]) - 1):
                    # output.write("\n")
        output.close()

In [5]:
target_1 = images[14]
target_2 = images[15]
current_dir = os.getcwd()
image_1_dir = os.path.join(current_dir, "images", target_1)
image_2_dir = os.path.join(current_dir, "images", target_2)
image_1_dir

'c:\\cv_assignment_5\\images\\KakaoTalk_20240530_125615705_14.jpg'

In [10]:

# match
target_1 = images[14]
target_2 = images[15]

image_path = os.path.join(image_dir, target_1)
image_1, corners_1, ids_1 = detect_markers(image_path)
image_path = os.path.join(image_dir, target_2)
image_2, corners_2, ids_2 = detect_markers(image_path)
(
    _matching_ids,
    matching_idx_1,
    matching_idx_2,
    matching_corners_1,
    matching_corners_2,
) = find_match_points(corners_1, ids_1, corners_2, ids_2)



current_dir = os.getcwd()
image_1_dir = os.path.join(current_dir, "images", target_1)
image_2_dir = os.path.join(current_dir, "images", target_2)

# Ensure the drive letter is capitalized
image_1_dir = image_1_dir[0].upper() + image_1_dir[1:]
image_2_dir = image_2_dir[0].upper() + image_2_dir[1:]

with open("match.txt", "w") as output:
    output.write(f"{image_1_dir}\n")
    output.write(f"{image_2_dir}\n")
    output.write(f"{len(matching_corners_1)}\n")
    output.write(
        " ".join(
            map(
                str,
                np.array(matching_idx_1).reshape(
                    -1,
                ),
            )
        )
        + "\n"
    )
    output.write(
        " ".join(
            map(
                str,
                np.array(matching_idx_2).reshape(
                    -1,
                ),
            )
        )
        + "\n"
    )

In [9]:
def write_match_file(target_1, target_2, idx1, idx2):
    image_path = os.path.join(image_dir, target_1)
    image_1, corners_1, ids_1 = detect_markers(image_path)
    image_path = os.path.join(image_dir, target_2)
    image_2, corners_2, ids_2 = detect_markers(image_path)
    (
        _matching_ids,
        matching_idx_1,
        matching_idx_2,
        matching_corners_1,
        matching_corners_2,
    ) = find_match_points(corners_1, ids_1, corners_2, ids_2)

    if _matching_ids == []:
        return
    
    current_dir = os.getcwd()
    image_1_dir = os.path.join(current_dir, "images", target_1)
    image_2_dir = os.path.join(current_dir, "images", target_2)

    # Ensure the drive letter is capitalized
    image_1_dir = image_1_dir[0].upper() + image_1_dir[1:]
    image_2_dir = image_2_dir[0].upper() + image_2_dir[1:]

    with open(f"./matches/match_{idx1}_{idx2}.txt", "w") as output:
        output.write(f"{image_1_dir}\n")
        output.write(f"{image_2_dir}\n")
        output.write(f"{len(matching_corners_1)}\n")
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_1).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_2).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        
        output.close()

for index1, image1 in enumerate(images):
    for index2, image2 in enumerate(images):
        if index1 == index2:
            continue
        write_match_file(image1, image2, index1, index2)

In [ ]:
def write_match_file(target_1, target_2, idx1, idx2):
    image_path = os.path.join(image_dir, target_1)
    image_1, corners_1, ids_1 = detect_markers(image_path)
    image_path = os.path.join(image_dir, target_2)
    image_2, corners_2, ids_2 = detect_markers(image_path)
    (
        _matching_ids,
        matching_idx_1,
        matching_idx_2,
        matching_corners_1,
        matching_corners_2,
    ) = find_match_points(corners_1, ids_1, corners_2, ids_2)

    if _matching_ids == []:
        return
    
    current_dir = os.getcwd()
    image_1_dir = os.path.join(current_dir, "images", target_1)
    image_2_dir = os.path.join(current_dir, "images", target_2)

    # Ensure the drive letter is capitalized
    image_1_dir = image_1_dir[0].upper() + image_1_dir[1:]
    image_2_dir = image_2_dir[0].upper() + image_2_dir[1:]

    with open(f"./matches/match_all.txt", "w") as output:
        output.write(f"{image_1_dir}\n")
        output.write(f"{image_2_dir}\n")
        output.write(f"{len(matching_corners_1)}\n")
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_1).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_2).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        
        output.close()

for index1, image1 in enumerate(images):
    for index2, image2 in enumerate(images):
        if index1 == index2:
            continue
        write_match_file(image1, image2, index1, index2)

In [12]:
def write_match_file(target_1, target_2, idx1, idx2):
    image_path = os.path.join(image_dir, target_1)
    image_1, corners_1, ids_1 = detect_markers(image_path)
    image_path = os.path.join(image_dir, target_2)
    image_2, corners_2, ids_2 = detect_markers(image_path)
    (
        _matching_ids,
        matching_idx_1,
        matching_idx_2,
        matching_corners_1,
        matching_corners_2,
    ) = find_match_points(corners_1, ids_1, corners_2, ids_2)

    if _matching_ids == []:
        print(f"{target_1} and {target_2} has no matching points")
        return
    
    current_dir = os.getcwd()
    image_1_dir = os.path.join(current_dir, "images", target_1)
    image_2_dir = os.path.join(current_dir, "images", target_2)

    # Ensure the drive letter is capitalized
    image_1_dir = image_1_dir[0].upper() + image_1_dir[1:]
    image_2_dir = image_2_dir[0].upper() + image_2_dir[1:]

    with open(f"./matches/match_all.txt", "a") as output:
        output.write(f"{image_1_dir}\n")
        output.write(f"{image_2_dir}\n")
        output.write(f"{len(matching_corners_1)}\n")
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_1).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        output.write(
            " ".join(
                map(
                    str,
                    np.array(matching_idx_2).reshape(
                        -1,
                    ),
                )
            )
            + "\n"
        )
        
        output.close()

for index1, image1 in enumerate(images):
    for index2, image2 in enumerate(images):
        if index1 == index2:
            continue
        write_match_file(image1, image2, index1, index2)

In [8]:
import os

print(os.getcwd())

c:\cv_assignment_5
